In [2]:
!python.exe -m pip install --upgrade pip

In [3]:
!pip install opencv-python matplotlib mediapipe

In [3]:
!pip install mediapipe

In [4]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [4]:
def mediapipe_detection(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    result=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,result

In [5]:
mp_holistic=mp.solutions.holistic
mp_drawing=mp.solutions.drawing_utils

In [6]:
def drawing_landmark(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [103]:
def save_keypoints(result):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in result.pose_landmarks.landmark]).flatten() if result.pose_landmarks else np.zeros(33*4)
    lh=np.array([[res.x,res.y,res.z,res.visibility] for res in result.left_hand_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(21*4)
    rh=np.array([[res.x,res.y,res.z,res.visibility] for res in result.right_hand_landmarks.landmark]).flatten() if result.right_hand_landmarks else np.zeros(21*4)         
    face=np.array([[res.x,res.y,res.z,res.visibility] for res in result.face_landmarks.landmark]).flatten() if result.face_landmarks else np.zeros(468*4) 
    return np.concatenate([pose,face,lh,rh])

In [8]:
actions=np.array(['hello','thanks','i love you'])

In [230]:
DATA_PATH=os.path.join('MP_DATA')
actions=np.array(['hello','thanks','i love you'])
sequence_lenght=30
no_sequence=30
for action in actions:
    for sequence in range (no_sequence):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass


In [231]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequence):
            for frame_num in range(sequence_lenght):
                ret,frame=cap.read()
                image,result=mediapipe_detection(frame,holistic)
                drawing_landmark(image,result)
                cv2.imshow('opencv feed',image)
                if frame_num==0:
                    cv2.putText(image,"Starting collection", (120,200), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image,'Collection starting for {} video number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow('opencv feed',image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'Collection starting for {} video number {}'.format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow('opencv feed',image)
                keypoints=save_keypoints(result)
                npy_path=os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)

                if cv2.waitKey(10) & 0xFF==ord('q'):
                    break


    cap.release()
    cv2.destroyAllWindows()
cap.release()
cv2.destroyAllWindows()

In [108]:
keypoints.shape

(2172,)

In [187]:
cap.release()
cv2.destroyAllWindows()

In [96]:
len(result.pose_landmarks.landmark)

33

In [9]:
!pip install scikit-learn

  Using cached scikit_learn-1.7.1-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.1-cp311-cp311-win_amd64.whl (8.9 MB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- --------------------------

In [122]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [190]:
label_map={label:num for num, label in enumerate(actions)}

In [232]:
sequences,labels=[],[]
for action in actions:
    for sequence in range(no_sequence):
        window=[]
        for frame_num in range(sequence_lenght):
            res=np.load(os.path.join(DATA_PATH,action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])


In [125]:
np.array(sequences).shape

(60, 15, 2172)

In [126]:
np.array(labels).shape

(60,)

In [233]:
x=np.array(sequences)

In [76]:
x.shape

(90, 30)

In [234]:
y=to_categorical(labels).astype(int)

In [13]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.05)

NameError: name 'train_test_split' is not defined

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [237]:
log_dir=os.path.join('Logs')
tb_callbacks=TensorBoard(log_dir=log_dir)

In [222]:
!pip install tensorboard

In [9]:
model=Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,2172)))
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

c:\Users\himan\Downloads\pinecone\.conda\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [239]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [240]:
model.fit(x_train,y_train,epochs=160,callbacks=[tb_callbacks])

Epoch 1/160
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - categorical_accuracy: 0.3307 - loss: 1.1620
Epoch 2/160
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 0.4384 - loss: 1.7916
Epoch 3/160
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - categorical_accuracy: 0.4501 - loss: 1.0382
Epoch 4/160
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - categorical_accuracy: 0.2153 - loss: 1.1267
Epoch 5/160
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - categorical_accuracy: 0.3190 - loss: 1.0123
Epoch 6/160
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - categorical_accuracy: 0.3288 - loss: 1.0774
Epoch 7/160
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 0.3503 - loss: 1.0050
Epoch 8/160
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - categorical_accuracy: 0.3229 - loss: 0.9531
Epoch 9/160
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 0.3738 - loss: 0.8509
Epoch 10/160
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - categorical_accuracy: 0.7279 - loss: 0.8123
Epoch 11/160
3/3 ━━━━━━━━━━━━━━━━━━━━ 0

In [12]:
res=model.predict(x_test)


NameError: name 'x_test' is not defined

In [244]:
y_test.shape

(5, 3)

In [245]:
actions[np.argmax(res[1])]

np.str_('thanks')

In [246]:
actions[np.argmax(y_test[1])]

np.str_('thanks')

In [241]:
model.save('action.h5')

In [11]:
model=Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,2172)))
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))
model.load_weights('action.h5')

In [205]:
from sklearn.metrics import accuracy_score

In [247]:
y_true=np.argmax(y_test,axis=1).tolist()
y_hat=np.argmax(res,axis=1).tolist()

In [228]:
accuracy_score(y_true,y_hat)

1.0

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [248]:
res[np.argmax(res)]

array([0., 1., 0.], dtype=float32)

In [250]:
sequence=[]
sentence=[]
cap=cv2.VideoCapture(0)
threshold=0.8
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
                ret,frame=cap.read()
                image,result=mediapipe_detection(frame,holistic)
                drawing_landmark(image,result)
                keypoints=save_keypoints(result)
                sequence.insert(0,keypoints)
                sequence=sequence[-30:]
                if len(sequence)==30:
                    res=model.predict(np.expand_dims(sequence,axis=0))[0]
                    print(actions[np.argmax(res)])
                    if res[np.argmax(res)]>threshold:
                        if len(sentence)>0:
                            if actions[np.argmax(res)]!=sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])
                    if len(sentence)>5:
                        sentence=sentence[-5:]
                cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
                cv2.putText(image,' '.join(sentence),(3,30),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2,cv2.LINE_AA)
                cv2.imshow('Opencv feed',image)
                if cv2.waitKey(10) & 0xFF==ord('q'):
                    break
        cap.release()
        cv2.destroyAllWindows()

SyntaxError: invalid non-printable character U+00A0 (1667433767.py, line 6)

In [ ]:
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,
                          min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image, result = mediapipe_detection(frame, holistic)
        drawing_landmark(image, result)

        keypoints = save_keypoints(result)  # Should be length 1662
        sequence.append(keypoints)
        sequence = sequence[-30:]  # Keep last 30 frames

        if len(sequence) == 30:
            input_seq = np.expand_dims(sequence, axis=0)  # Shape: (1, 30, 1662)
            res = model.predict(input_seq)[0]
            action = actions[np.argmax(res)]
            print(action)

            if res[np.argmax(res)] > threshold:
                if len(sentence) >0:
                   if actions[np.argmax(res)] != sentence[-1]:
                     sentence.append(actions[np.argmax(res)])
            if len(sentence) > 5:
                sentence = sentence[-5:]

        # Display box and sentence
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(actions[np.argmax(res)]), (3, 30),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)

        cv2.imshow('Opencv feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0